# 1NN против RandomForest


В этом задании будет использоваться датасет `digits` из `sklearn.datasets`. **Оставьте последние 25% объектов для контроля качества**, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

**Задание 1**

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

**Задание 2**

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

Как отправить
Когда работа будет готова, вы можете загрузить файлы для каждой части задания на вкладке 'Мои работы'.

In [82]:
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [27]:
def write_answer(value, filename):
    """
    Функция для записи ответов.
    * value - значение, которое нужно записать в файл
    * filename - имя файла
    
    """
    with open(filename+".txt", "w") as file:
        file.write(str(value))

In [91]:
# Загрузка данных
data, target = load_digits(return_X_y=True, as_frame=True)

In [92]:
data.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [93]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=int(0.25*data.shape[0]), shuffle=False)

In [94]:
# X_train = data.loc[0:int(0.75*data.shape[0])]
# y_train = target.loc[0:int(0.75*data.shape[0])]

# X_test = data.loc[int(0.75*data.shape[0])+1:]
# y_test = target.loc[int(0.75*data.shape[0]+1):]

In [95]:
X_train.tail()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
1343,0.0,0.0,0.0,1.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,15.0,13.0,0.0,0.0
1344,0.0,0.0,11.0,16.0,16.0,7.0,0.0,0.0,0.0,2.0,...,4.0,1.0,0.0,0.0,10.0,16.0,16.0,16.0,16.0,10.0
1345,0.0,0.0,0.0,14.0,14.0,1.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,12.0,16.0,14.0,1.0,0.0
1346,0.0,0.0,8.0,16.0,16.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,16.0,16.0,7.0,0.0,0.0
1347,0.0,0.0,7.0,16.0,16.0,14.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,12.0,16.0,16.0,6.0,0.0,0.0


In [96]:
X_test.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
1348,0.0,0.0,12.0,16.0,16.0,5.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,13.0,10.0,0.0,0.0,0.0,0.0
1349,0.0,0.0,5.0,15.0,16.0,15.0,1.0,0.0,0.0,10.0,...,12.0,0.0,0.0,0.0,5.0,15.0,16.0,14.0,3.0,0.0
1350,0.0,0.0,10.0,16.0,16.0,10.0,1.0,0.0,0.0,4.0,...,15.0,1.0,0.0,0.0,11.0,16.0,16.0,15.0,4.0,0.0
1351,0.0,0.0,0.0,1.0,13.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,9.0,0.0,0.0
1352,0.0,0.0,0.0,14.0,14.0,1.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,14.0,16.0,14.0,0.0,0.0


In [97]:
# Проверка
print(data.shape[0])
print(X_train.shape[0])
print(X_test.shape[0])
X_train.shape[0]+X_test.shape[0] == data.shape[0]

1797
1348
449


True

In [98]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
one_neighbour = KNeighborsClassifier(n_neighbors=1, p=2)
one_neighbour.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [99]:
# доля верных ответов на обучающей выборке
accuracy_score(y_true=y_train, y_pred=one_neighbour.predict(X_train))

1.0

In [100]:
# доля верных ответов на тестовой выборке
accuracy_1nn = accuracy_score(y_true=y_test, y_pred=one_neighbour.predict(X_test))

In [101]:
accuracy_1nn

0.9621380846325167

In [102]:
write_answer(value=1-accuracy_1nn, filename='randomForest_vs_1nn_1')

In [103]:
rf_clf = RandomForestClassifier(n_estimators=1000)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [104]:
# доля верных ответов на обучающей выборке
accuracy_score(y_true=y_train, y_pred=rf_clf.predict(X_train))

1.0

In [105]:
# доля верных ответов на тестовой выборке
accuracy_rf = accuracy_score(y_true=y_test, y_pred=rf_clf.predict(X_test))
accuracy_rf

0.933184855233853

In [106]:
write_answer(value=1-accuracy_rf, filename='randomForest_vs_1nn_2')